# Kindle Note Caster


### problématique:

kindle fournit un csv ou bien un pdf, pas pratique pour relire ses notes et en faire un seul document facile à lire: je les cast en Microsoft word

### détails techniques:

https://anaconda.org/conda-forge/python-docx

https://regex101.com/

https://docs.python.org/3/library/re.html


### améliorations globales possibles:

annotation + mettre le nom du titre du livre en haut

gestion du chemin absolu ( pour l'instant doit etre dans le dossier)

gestion différamment des surlignements et des notes

gestion des photos sur Kindle : contacter Amazon?

### Là ou j'en suis:

faire la fonction de routage des docs avec les mots clés @@:
    - dans le else

comprendre pourquoi le csv n'est plus bien lu par python ( tout dans la colonne 1)
    - investiguer error bad lines parameters
    - delimiter
    -le fait d'ouvrir le csv avec excel auparant? 

revoir le nommage des colonnes pour respecter celles d'amazon

In [1]:
### import

import pandas as pd
import numpy as np
import docx
import json
import re

In [2]:
### Nom des chemins:

i_chemin_du_csv= "A.csv"
o_chemin_du_word= "The_Customer_Success_Professionals_Handbook.docx"


In [3]:
### fonctions:

def ajouter_surlignement(surlignement):
    global o_texte
    if (str(surlignement) not in List_mot_a_enlever):
        o_texte = o_texte +'\n\n'+ str(surlignement)

def ajouter_note(note):
    global o_texte
    if (str(note) not in List_mot_a_enlever):
        o_texte = o_texte +' => '+ str(note)
        
def num_emplacement(emplacement):
    return int(emplacement[12:])

def check_hashtag(note):
    if '@@' in note:
        return True
    return False

def extract_keyword(text):
    # example: sales de "##sales commentaire1 commentaire2"
    pattern=re.compile(r"""(?<=@@)([A-ù]+)(?= )""")
    match = re.search(pattern, text)
    if match:
        return(match[0].lower())
    print("error: keyword  not found")
    
extract_keyword("@@SAles commentaire1 commentaire2")

'sales'

In [4]:
### script
df=pd.read_csv(i_chemin_du_csv, error_bad_lines=False) #,delimiter=',')
df

b'Skipping line 396: expected 6 fields, saw 7\n'


,,Vos notes Kindle pour :,Unnamed: 1,Unnamed: 2,;
THE HARDWARE STARTUP: BUILDING YOUR PRODUCT,BUSINESS,AND BRAND (ENGLISH EDITION),NaN,NaN,;
de Renee DiResta,Brady Forrest,Ryan Vinyard,NaN,NaN,;
Aperçu instantané Kindle gratuit :,NaN,NaN,;,NaN,NaN
https://amzn.eu/dkzEz6M,NaN,NaN,;,NaN,NaN
----------------------------------------------,NaN,NaN,;,NaN,NaN
...,...,...,...,...,...
"Surlignement (Jaune),""Emplacement 6626"","""",""Ultimately, it’s the importer’s responsibility to get all of the papers filed and duties paid correctly. A mistake can result in goods being held at the dock, fines, audits, or increased scrutiny of future shipments"";",NaN,NaN,NaN,NaN,NaN
"Surlignement (Jaune),""Emplacement 6629"","""",""Because of the complexity, many importers hire a customs broker to take care of the paperwork and filings for them."";",NaN,NaN,NaN,NaN,NaN
"Surlignement (Jaune),""Emplacement 6725"","""",""The Cellscope team hired a regulatory consultant to help them understand the myriad rules that applied to their product. They also hired a full-time quality engineer, whose job entails making sure that the quality assurance system"";",NaN,NaN,NaN,NaN,NaN
"Note,""Emplacement 6727"","""",""Importance du quality engineer dans un marché regulé"";",NaN,NaN,NaN,NaN,NaN


In [5]:
o_texte = ""
List_mot_a_enlever=['nan','Annotation'] # ce son en fait les titres des colonnes

In [6]:
#load file_mapper
file_mapper = json.load(open('file_mapper.json'))
#rename columns Type d'annotation,"Emplacement","Suivi ?","Annotation"
df.columns = ['Type','Emplacement','unknown','texte']

In [9]:
df['texte'].apply(ajouter_surlignement)
"""
for i in df.index: 
    print(df['texte'][i] )
    if df['Type'][i]=='Surlignement (Jaune)':
        ajouter_surlignement(df['texte'][i])
        
    elif df['Type'][i]=='Note':
        print('here')
        #vérifie que la note est liée au surlignement précédent et qu'il n'y a pas de ##
        if (num_emplacement(df['Emplacement'][i])==num_emplacement(df['Emplacement'][i-1])):
            if not check_hashtag(df['texte'][i]):
                ajouter_note(df['texte'][i])
                
            else:
                # il y a bien un marquage de rediretion de notes: il faut placer la note dans le bon fichier
                # ce mapping (le bon endroit des fichiers) est stocké dans un dictionnaire dans un fichier texte,
                # celui ci est documenté dans le README
                # ajouter dans le fichier concerné la note + l'emplacement précédent
                #gestion de l'écriture dans les bons fichiers du dossier Lecture
                print(extract_keyword(df['texte'][i]))
                if extract_keyword(df['texte'][i]) in file_mapper.keys:
                    print(df['texte'][i], file_mapper[extract_keyword(df['texte'][i])] )
                
                
                
                

                
                
                
        else:        
            ajouter_surlignement(df['texte'][i]+"note")
# pas de else: les autres cas sont les résidus de texte en haut"""

'\nfor i in df.index: \n    print(df[\'texte\'][i] )\n    if df[\'Type\'][i]==\'Surlignement (Jaune)\':\n        ajouter_surlignement(df[\'texte\'][i])\n        \n    elif df[\'Type\'][i]==\'Note\':\n        print(\'here\')\n        #vérifie que la note est liée au surlignement précédent et qu\'il n\'y a pas de ##\n        if (num_emplacement(df[\'Emplacement\'][i])==num_emplacement(df[\'Emplacement\'][i-1])):\n            if not check_hashtag(df[\'texte\'][i]):\n                ajouter_note(df[\'texte\'][i])\n                \n            else:\n                # il y a bien un marquage de rediretion de notes: il faut placer la note dans le bon fichier\n                # ce mapping (le bon endroit des fichiers) est stocké dans un dictionnaire dans un fichier texte,\n                # celui ci est documenté dans le README\n                # ajouter dans le fichier concerné la note + l\'emplacement précédent\n                #gestion de l\'écriture dans les bons fichiers du dossier Lect

In [10]:
mydoc = docx.Document()
mydoc.add_paragraph(o_texte)
mydoc.save(o_chemin_du_word)
